<a href="https://colab.research.google.com/github/Leomuch/projek_analisis_data_dbs/blob/main/Copy_of_%5BClustering%5D_Submission_Akhir_BMLP_Muchlas_Andrey_Pahlevi_(Updated).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1. Perkenalan Dataset**


Tahap pertama, Anda harus mencari dan menggunakan dataset **tanpa label** dengan ketentuan sebagai berikut:

1. **Sumber Dataset**:  
   Dataset dapat diperoleh dari berbagai sumber, seperti public repositories (*Kaggle*, *UCI ML Repository*, *Open Data*) atau data primer yang Anda kumpulkan sendiri.
   
2. **Ketentuan Dataset**:
   - **Tanpa label**: Dataset tidak boleh memiliki label atau kelas.
   - **Jumlah Baris**: Minimal 1000 baris untuk memastikan dataset cukup besar untuk analisis yang bermakna.
   - **Tipe Data**: Harus mengandung data **kategorikal** dan **numerikal**.
     - *Kategorikal*: Misalnya jenis kelamin, kategori produk.
     - *Numerikal*: Misalnya usia, pendapatan, harga.

3. **Pembatasan**:  
   Dataset yang sudah digunakan dalam latihan clustering (seperti customer segmentation) tidak boleh digunakan.

# **2. Import Library**

Pada tahap ini, Anda perlu mengimpor beberapa pustaka (library) Python yang dibutuhkan untuk analisis data dan pembangunan model machine learning.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly as py
import plotly.graph_objs as go
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans

# **3. Memuat Dataset**

Pada tahap ini, Anda perlu memuat dataset ke dalam notebook. Jika dataset dalam format CSV, Anda bisa menggunakan pustaka pandas untuk membacanya. Pastikan untuk mengecek beberapa baris awal dataset untuk memahami strukturnya dan memastikan data telah dimuat dengan benar.

Jika dataset berada di Google Drive, pastikan Anda menghubungkan Google Drive ke Colab terlebih dahulu. Setelah dataset berhasil dimuat, langkah berikutnya adalah memeriksa kesesuaian data dan siap untuk dianalisis lebih lanjut.

In [2]:
women_in_data_science = "UnlabeledWiDS2021.csv"
df = pd.read_csv(women_in_data_science)

# **4. Exploratory Data Analysis (EDA)**

Pada tahap ini, Anda akan melakukan **Exploratory Data Analysis (EDA)** untuk memahami karakteristik dataset. EDA bertujuan untuk:

1. **Memahami Struktur Data**
   - Tinjau jumlah baris dan kolom dalam dataset.  
   - Tinjau jenis data di setiap kolom (numerikal atau kategorikal).

2. **Menangani Data yang Hilang**  
   - Identifikasi dan analisis data yang hilang (*missing values*). Tentukan langkah-langkah yang diperlukan untuk menangani data yang hilang, seperti pengisian atau penghapusan data tersebut.

Tujuan dari EDA adalah untuk memperoleh wawasan awal yang mendalam mengenai data dan menentukan langkah selanjutnya dalam analisis atau pemodelan.

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10234 entries, 0 to 10233
Columns: 180 entries, Unnamed: 0 to solid_tumor_with_metastasis
dtypes: float64(156), int64(18), object(6)
memory usage: 14.1+ MB


In [5]:
df.isna().sum()

,0
Unnamed: 0,0
encounter_id,0
hospital_id,0
age,0
bmi,1015
...,...
hepatic_failure,0
immunosuppression,0
leukemia,0
lymphoma,0


In [25]:
for i , (col, dtype) in enumerate(df.dtypes.items(), start=1):
    print(f"{i}. {col}: {dtype}")

1. age: int64
2. bmi: float64
3. elective_surgery: int64
4. ethnicity: int64
5. gender: int64
6. height: float64
7. hospital_admit_source: int64
8. icu_admit_source: int64
9. icu_id: int64
10. icu_stay_type: int64
11. icu_type: int64
12. pre_icu_los_days: float64
13. readmission_status: int64
14. weight: float64
15. albumin_apache: float64
16. apache_2_diagnosis: float64
17. apache_3j_diagnosis: float64
18. apache_post_operative: int64
19. arf_apache: int64
20. bilirubin_apache: float64
21. bun_apache: float64
22. creatinine_apache: float64
23. fio2_apache: float64
24. gcs_eyes_apache: float64
25. gcs_motor_apache: float64
26. gcs_unable_apache: float64
27. gcs_verbal_apache: float64
28. glucose_apache: float64
29. heart_rate_apache: float64
30. hematocrit_apache: float64
31. intubated_apache: int64
32. map_apache: float64
33. paco2_apache: float64
34. paco2_for_ph_apache: float64
35. pao2_apache: float64
36. ph_apache: float64
37. resprate_apache: float64
38. sodium_apache: float64
39

In [7]:
df.duplicated().sum()

np.int64(0)

In [8]:
df.describe()

,Unnamed: 0,encounter_id,hospital_id,age,bmi,elective_surgery,height,icu_id,pre_icu_los_days,readmission_status,...,h1_arterial_po2_min,h1_pao2fio2ratio_max,h1_pao2fio2ratio_min,aids,cirrhosis,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis
count,10234.000000,10234.000000,10234.000000,10234.000000,9219.000000,10234.000000,9933.000000,10234.000000,10234.000000,10234.0,...,1928.000000,1422.000000,1422.000000,10234.000000,10234.000000,10234.000000,10234.000000,10234.000000,10234.000000,10234.000000
mean,5117.500000,140498.779461,10092.391929,62.853625,29.116463,0.200117,169.276414,677.823725,0.831339,0.0,...,156.767998,254.687589,246.255826,0.000977,0.012019,0.010064,0.022572,0.006547,0.004104,0.017686
std,2954.445661,3182.455522,57.008062,17.851661,8.342873,0.400107,10.805701,304.116531,2.411936,0.0,...,105.623078,132.087636,130.745488,0.031245,0.108975,0.099821,0.148541,0.080651,0.063934,0.131814
min,1.000000,135000.000000,10001.000000,18.000000,14.900000,0.000000,137.000000,82.000000,-0.208333,0.0,...,30.000000,42.620000,41.417000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2559.250000,137742.250000,10043.000000,52.000000,23.700000,0.000000,162.500000,451.000000,0.032639,0.0,...,79.000000,151.400000,144.050000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,5117.500000,140503.500000,10085.000000,65.000000,27.600000,0.000000,170.000000,687.000000,0.134028,0.0,...,118.000000,234.000000,224.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,7675.750000,143252.750000,10145.000000,76.000000,32.600000,0.000000,177.800000,962.000000,0.430382,0.0,...,206.075000,345.525000,336.525000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,10234.000000,146000.000000,10199.000000,96.000000,69.944000,1.000000,195.600000,1111.000000,65.945833,0.0,...,518.060000,672.824000,651.656000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


### Hasil Analisis Awal Dataset
- Dataset memiliki 10234 baris dan 180 kolom.
- Tidak ada data yang duplikat.
- Mayoritas kolom adalah numerik (156 float, 18 integer), dengan 6 kolom kategorikal (object).
- Ada 159 kolom yang memiliki missing values, adapun beberapa kolom yang terbanyak memiliki missing value:
  - `h1_pao2fio2ratio_min` -> 8812 missing values
  - `h1_pao2fio2ratio_max` -> 8812 missing values
  - `h1_arterial_ph_max`  -> 8401 missing values
- Statistik Deskriptif:
  - Fitur `age`: Rata-rata 62.85 tahun, rentang 18 - 96 tahun.
  - Fitur `bmi`: Rata-rata 29.11, beberapa outlier hingga 69.94.
  - Banyak fitur berupa binary (0/1) untuk kondisi medis, seperti `cirrhosis`, `leukemia`, `lymphoma`.

# **5. Data Preprocessing**

Pada tahap ini, data preprocessing adalah langkah penting untuk memastikan kualitas data sebelum digunakan dalam model machine learning. Data mentah sering kali mengandung nilai kosong, duplikasi, atau rentang nilai yang tidak konsisten, yang dapat memengaruhi kinerja model. Oleh karena itu, proses ini bertujuan untuk membersihkan dan mempersiapkan data agar analisis berjalan optimal.

Berikut adalah tahapan-tahapan yang bisa dilakukan, tetapi **tidak terbatas** pada:
1. Menghapus atau Menangani Data Kosong (Missing Values)
2. Menghapus Data Duplikat
3. Normalisasi atau Standarisasi Fitur
4. Deteksi dan Penanganan Outlier
5. Encoding Data Kategorikal
6. Binning (Pengelompokan Data)

Cukup sesuaikan dengan karakteristik data yang kamu gunakan yah.

### 1. Menghapus Kolom yang tidak relevan

In [9]:
df.drop(columns=["Unnamed: 0", "encounter_id", "hospital_id"], errors="ignore", inplace=True)

### 2. Memisahkan kolom Numerik dan Kategorikal dan menanganinya
numerik ditangani dengan mengisi nilai mean sedangkan kategorikal ditangani dengan mengisi nilai modus

In [10]:
numeric_cols = df.select_dtypes(include=['number']).columns
categorical_cols = df.select_dtypes(include=['object']).columns
df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].mean())
df[categorical_cols] = df[categorical_cols].fillna(df[categorical_cols].mode().iloc[0])

Dataset setelah menghapus kolom yang tidak relevan dan Missing Value setelah ditangani

In [11]:
missing_after = df.isna().sum().sum()
missing_after, df.shape

(np.int64(0), (10234, 177))

### 3. Encoding Data Kategorikal

In [12]:
label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le

### 4. Normalisasi Data dengan StandardScaler

Untuk memastikan semua fitur memiliki skala yang seragam, kita menggunakan **StandardScaler** dari `sklearn.preprocessing`. StandardScaler bekerja dengan cara:

1. **Menghitung mean (μ) dan standar deviasi (σ) dari setiap fitur**  
2. **Mengonversi nilai setiap fitur ke skala standar menggunakan rumus berikut:**  
   \[
   X_{scaled} = \frac{X - \mu}{\sigma}
   \]
   Di mana:
   - \(X\) adalah nilai asli dari fitur  
   - \(\mu\) adalah rata-rata dari fitur tersebut  
   - \(\sigma\) adalah standar deviasi dari fitur tersebut  

### Implementasi Kode:
```python
# Inisialisasi StandardScaler
scaler = StandardScaler()

# Melakukan transformasi pada dataset
df_scaled = scaler.fit_transform(df)


In [13]:
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df)

In [14]:
selector = VarianceThreshold(threshold=0.01)
df_selected = selector.fit_transform(df)
print(f"Jumlah fitur sebelum seleksi: {df.shape[1]}")
print(f"Jumlah fitur setelah seleksi: {df_selected.shape[1]}")

Jumlah fitur sebelum seleksi: 177
Jumlah fitur setelah seleksi: 167


In [15]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df_selected)

In [16]:
pca = PCA(n_components=2)
df_pca = pca.fit_transform(df_scaled)
pca_variance = pca.explained_variance_ratio_
df_pca.shape, pca_variance

((10234, 2), array([0.0910544 , 0.04821856]))

In [23]:
pca_components = pd.DataFrame(pca.components_, columns=df.columns, index=['PC1', 'PC2'])
print(pca_components.T)

                                  PC1       PC2
age                         -0.038680  0.033411
bmi                          0.009907  0.036219
elective_surgery            -0.020875 -0.124063
ethnicity                   -0.009629 -0.023880
gender                       0.017567  0.008522
...                               ...       ...
hepatic_failure             -0.018435  0.005242
immunosuppression           -0.019739  0.009846
leukemia                    -0.012755  0.011512
lymphoma                    -0.010362 -0.000584
solid_tumor_with_metastasis -0.012502  0.003525

[177 rows x 2 columns]


In [24]:
# Mengambil 10 fitur dengan kontribusi terbesar di PC1 dan PC2
top_10_pc1 = pca_components.loc['PC1', :].abs().nlargest(10).index
top_10_pc2 = pca_components.loc['PC2', :].abs().nlargest(10).index

# Menampilkan fitur-fitur dengan kontribusi terbesar
print("Top 10 fitur yang berkontribusi terbesar di PC1:")
print(top_10_pc1)

print("\nTop 10 fitur yang berkontribusi terbesar di PC2:")
print(top_10_pc2)

Top 10 fitur yang berkontribusi terbesar di PC1:
Index(['h1_mbp_min', 'h1_mbp_noninvasive_min', 'h1_diasbp_min',
       'h1_diasbp_noninvasive_min', 'h1_sysbp_min', 'h1_mbp_max',
       'h1_sysbp_noninvasive_min', 'h1_mbp_noninvasive_max', 'h1_sysbp_max',
       'h1_sysbp_noninvasive_max'],
      dtype='object')

Top 10 fitur yang berkontribusi terbesar di PC2:
Index(['d1_bun_max', 'bun_apache', 'd1_creatinine_max', 'creatinine_apache',
       'd1_bun_min', 'd1_creatinine_min', 'h1_bun_min', 'h1_bun_max',
       'h1_creatinine_max', 'h1_creatinine_min'],
      dtype='object')


In [19]:
selected_features = list(set(top_10_pc1) | set(top_10_pc2))
selected_features

['h1_mbp_noninvasive_min',
 'h1_sysbp_min',
 'h1_bun_max',
 'h1_sysbp_noninvasive_max',
 'h1_creatinine_min',
 'creatinine_apache',
 'h1_diasbp_noninvasive_min',
 'd1_creatinine_min',
 'h1_sysbp_noninvasive_min',
 'h1_mbp_min',
 'h1_diasbp_min',
 'h1_creatinine_max',
 'h1_sysbp_max',
 'd1_bun_min',
 'h1_bun_min',
 'h1_mbp_noninvasive_max',
 'd1_creatinine_max',
 'bun_apache',
 'd1_bun_max',
 'h1_mbp_max']

In [20]:
scaler = StandardScaler()
df_scaled_sf = scaler.fit_transform(selected_features)

ValueError: could not convert string to float: 'h1_mbp_noninvasive_min'

In [ ]:
inertia = []
for n in range(1, 11):
    algorithm = KMeans(n_clusters = n, random_state = 111, init = 'k-means++', n_init = 10, max_iter = 300,
                    tol = 0.0001, algorithm='elkan')
    algorithm.fit(df_pca)
    inertia.append(algorithm.inertia_)

In [ ]:
plt.figure(1 , figsize = (15 ,6))
plt.plot(np.arange(1 , 11) , inertia , 'o')
plt.plot(np.arange(1 , 11) , inertia , '-' , alpha = 0.5)
plt.xlabel('Number of Clusters') , plt.ylabel('Inertia')
plt.show()

In [ ]:
algorithm = (KMeans(n_clusters = 4 ,init='k-means++', n_init = 10 ,max_iter=300,
                        tol=0.0001,  random_state= 111  , algorithm='elkan') )

algorithm.fit(df_pca)
labels1 = algorithm.labels_
centroids1 = algorithm.cluster_centers_

In [ ]:
h = 0.02
x_min, x_max = df_pca[:, 0].min() - 1, df_pca[:, 0].max() + 1
y_min, y_max = df_pca[:, 1].min() - 1, df_pca[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
Z = algorithm.predict(np.c_[xx.ravel(), yy.ravel()])

In [ ]:
plt.figure(1, figsize=(15, 7))
plt.clf()

# Bentuk ulang hasil prediksi agar sesuai dengan grid
Z = Z.reshape(xx.shape)

# Plot decision boundary
plt.imshow(Z, interpolation='nearest',
           extent=(xx.min(), xx.max(), yy.min(), yy.max()),
           cmap=plt.cm.Pastel2, aspect='auto', origin='lower')

# Plot data points
plt.scatter(df_pca[:, 0], df_pca[:, 1], c=labels1, cmap='viridis', edgecolors='k', alpha=0.7)

# Plot centroids
plt.scatter(centroids1[:, 0], centroids1[:, 1], c='red', marker='X', s=200, label="Centroids")

plt.xlabel("Principal Component 1")
plt.ylabel("Principal Component 2")
plt.title("KMeans Clustering with Decision Boundary")
plt.legend()
plt.show()


In [ ]:
score = silhouette_score(df_pca, labels1)
print(f"Silhouette Score: {score:.4f}")

# **6. Pembangunan Model Clustering**

## **a. Pembangunan Model Clustering**

Pada tahap ini, Anda membangun model clustering dengan memilih algoritma yang sesuai untuk mengelompokkan data berdasarkan kesamaan. Berikut adalah **rekomendasi** tahapannya.
1. Pilih algoritma clustering yang sesuai.
2. Latih model dengan data menggunakan algoritma tersebut.

In [ ]:
#Type your code here

## **b. Evaluasi Model Clustering**

Untuk menentukan jumlah cluster yang optimal dalam model clustering, Anda dapat menggunakan metode Elbow atau Silhouette Score.

Metode ini membantu kita menemukan jumlah cluster yang memberikan pemisahan terbaik antar kelompok data, sehingga model yang dibangun dapat lebih efektif. Berikut adalah **rekomendasi** tahapannya.
1. Gunakan Silhouette Score dan Elbow Method untuk menentukan jumlah cluster optimal.
2. Hitung Silhouette Score sebagai ukuran kualitas cluster.

In [ ]:
#Type your code here

## **c. Feature Selection (Opsional)**

Silakan lakukan feature selection jika Anda membutuhkan optimasi model clustering. Jika Anda menerapkan proses ini, silakan lakukan pemodelan dan evaluasi kembali menggunakan kolom-kolom hasil feature selection. Terakhir, bandingkan hasil performa model sebelum dan sesudah menerapkan feature selection.

In [ ]:
#Type your code here

## **d. Visualisasi Hasil Clustering**

Setelah model clustering dilatih dan jumlah cluster optimal ditentukan, langkah selanjutnya adalah menampilkan hasil clustering melalui visualisasi.

Berikut adalah **rekomendasi** tahapannya.
1. Tampilkan hasil clustering dalam bentuk visualisasi, seperti grafik scatter plot atau 2D PCA projection.

In [ ]:
#Type your code here

## **e. Analisis dan Interpretasi Hasil Cluster**

### Interpretasi Target

**Tutorial: Melakukan Inverse Transform pada Data Target Setelah Clustering**

Setelah melakukan clustering dengan model **KMeans**, kita perlu mengembalikan data yang telah diubah (normalisasi, standarisasi, atau label encoding) ke bentuk aslinya. Berikut adalah langkah-langkahnya.

---

**1. Tambahkan Hasil Label Cluster ke DataFrame**
Setelah mendapatkan hasil clustering, kita tambahkan label cluster ke dalam DataFrame yang telah dinormalisasi.

```python
df_normalized['Cluster'] = model_kmeans.labels_
```

Lakukan Inverse Transform pada feature yang sudah dilakukan Labelisasi dan Standararisasi. Berikut code untuk melakukannya:
label_encoder.inverse_transform(X_Selected[['Fitur']])

Lalu masukkan ke dalam kolom dataset asli atau membuat dataframe baru
```python
df_normalized['Fitur'] = label_encoder.inverse_transform(df_normalized[['Fitur']])
```
Masukkan Data yang Sudah Di-Inverse ke dalam Dataset Asli atau Buat DataFrame Baru
```python
df_original['Fitur'] = df_normalized['Fitur']
```

In [ ]:
# Type your code here


### Inverse Data Jika Melakukan Normalisasi/Standardisasi

Inverse Transform untuk Data yang Distandarisasi
Jika data numerik telah dinormalisasi menggunakan StandardScaler atau MinMaxScaler, kita bisa mengembalikannya ke skala asli:
```python
df_normalized[['Fitur_Numerik']] = scaler.inverse_transform(df_normalized[['Fitur_Numerik']])
```

In [ ]:
# Type your code here

Setelah melakukan clustering, langkah selanjutnya adalah menganalisis karakteristik dari masing-masing cluster berdasarkan fitur yang tersedia.

Berikut adalah **rekomendasi** tahapannya.
1. Analisis karakteristik tiap cluster berdasarkan fitur yang tersedia (misalnya, distribusi nilai dalam cluster).
2. Berikan interpretasi: Apakah hasil clustering sesuai dengan ekspektasi dan logika bisnis? Apakah ada pola tertentu yang bisa dimanfaatkan?

In [ ]:
# Type your code here

Tulis hasil interpretasinya di sini.
1. Cluster 1:
2. Cluster 2:
3. Cluster 3:

# Contoh interpretasi [TEMPLATE]
# Analisis Karakteristik Cluster dari Model KMeans

Berikut adalah analisis karakteristik untuk setiap cluster yang dihasilkan dari model KMeans.

## Cluster 1:
- **Rata-rata Annual Income (k$):** 48,260  
- **Rata-rata Spending Score (1-100):** 56.48  
- **Analisis:** Cluster ini mencakup pelanggan dengan pendapatan tahunan menengah dan tingkat pengeluaran yang cukup tinggi. Pelanggan dalam cluster ini cenderung memiliki daya beli yang moderat dan mereka lebih cenderung untuk membelanjakan sebagian besar pendapatan mereka.

## Cluster 2:
- **Rata-rata Annual Income (k$):** 86,540  
- **Rata-rata Spending Score (1-100):** 82.13  
- **Analisis:** Cluster ini menunjukkan pelanggan dengan pendapatan tahunan tinggi dan pengeluaran yang sangat tinggi. Pelanggan di cluster ini merupakan kelompok premium dengan daya beli yang kuat dan cenderung mengeluarkan uang dalam jumlah besar untuk produk atau layanan.

## Cluster 3:
- **Rata-rata Annual Income (k$):** 87,000  
- **Rata-rata Spending Score (1-100):** 18.63  
- **Analisis:** Cluster ini terdiri dari pelanggan dengan pendapatan tahunan yang tinggi tetapi pengeluaran yang rendah. Mereka mungkin memiliki kapasitas finansial yang baik namun tidak terlalu aktif dalam berbelanja. Ini bisa menunjukkan bahwa mereka lebih selektif dalam pengeluaran mereka atau mungkin lebih cenderung untuk menyimpan uang.

# **7. Mengeksport Data**

Simpan hasilnya ke dalam file CSV.